### json 불러와서 저장하기

In [1]:
import numpy as np
import pandas as pd
import json
import jpype
from bs4 import BeautifulSoup as bs

data_path = "data"
data_path = "data"

json_train=[]
for i in range(0,18):
    json_file = json.loads(open("%s/%d.json" % (data_path, i)).read())
    json_train += json_file["articles"]

### json에서 author, author 형태소, forumid 추출

In [2]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "author": article["author"],
            "author_pos": ["%s_%s" % (first, second) for first, second in mecab.pos(article["author"])],
            "forumid": article["forumid"],
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


,author,author_pos,forumid,istroll
0,세부이장,"[세부_NNG, 이장_NNG]",마을회관밭,False
1,관리자,[관리자_NNG],잡담밭,False
2,snoopy,[snoopy_SL],잡담밭,False
3,ISCARIOT,[ISCARIOT_SL],정치사회밭,False
4,1234,[1234_SN],연예인밭,False
5,호놀룰루,[호놀룰루_NNP],정치사회밭,False
6,영양죽,"[영_NNG, 양죽_NNG]",잡담밭,False
7,영양죽,"[영_NNG, 양죽_NNG]",연예인밭,False
8,fakebook,[fakebook_SL],잡담밭,False
9,그나,[그나_MAJ],잡담밭,False


### forumid와 author을 label encoder로 숫자로 바꿔줌

In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

labeled_train["forumid"] = le.fit_transform(labeled_train["forumid"])
labeled_train["author"] = le.fit_transform(labeled_train["author"])

labeled_train

,author,author_pos,forumid,istroll
0,15848,"[세부_NNG, 이장_NNG]",16,False
1,5653,[관리자_NNG],31,False
2,3752,[snoopy_SL],31,False
3,1258,[ISCARIOT_SL],32,False
4,136,[1234_SN],23,False
5,26299,[호놀룰루_NNP],32,False
6,18849,"[영_NNG, 양죽_NNG]",31,False
7,18849,"[영_NNG, 양죽_NNG]",23,False
8,2693,[fakebook_SL],31,False
9,6140,[그나_MAJ],31,False


###bag of words

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)

    train_data_features = vectorizer.fit_transform((labeled_train[col_name].apply(lambda x:" ".join(x)))).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    labeled_train = pd.concat([labeled_train, pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)],axis=1)
    
    return labeled_train

##LDA

In [19]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    for i in range(0,num):
        if i%10000 == 0: print(i)
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = index
    
    train = pd.concat([train,df],axis=1)
        
    return train

In [20]:
max_features = 1000
%time labeled_train = make_lda(labeled_train, 5000, 1000, "author_pos")

Make Lda..
0


ValueError: max() arg is an empty sequence

### predictor, model 생성

In [ ]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['author_pos', 'author', label])
#pre = ['author','forumid']

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [ ]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

## Result

 * author를 Label Encoder로 변환: 0.67331271058
 * author의 morphs를 Bag of Words(feature = 1,000 개)로 변환: 0.733333689652
 * author의 pos를 Bag of Words(feature = 1,000 개)로 변환: 0.738635749417
 
 
 * author의 pos를 LDA로 변환 (num_topics = 1,000, keep_n = 5,000) : 0.636041209245

In [13]:
import pickle

pre = labeled_train.columns.drop(['author_pos', 'author'])

pickle.dump(labeled_train[pre], open("author, forum_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)

2
